In [2]:
from datetime import datetime
import ccxt
import json

In [ ]:

#Connexion à l'API
kucoin = ccxt.kucoin({
    "apiKey":"62a05a9ccf9fd60001e2ff9e",
    "secret":"578687f6-5420-46d7-9b7a-26a737713cfb",
    "password":"3FiF$4sR6p5Y9fPnSNijL?HaGcd@A7"
})
def getSolde():
    global kucoin
    try:
        for coin in kucoin.fetchBalance()['info']['data'] :
            if coin['currency']=='USDT' :
                return float(coin['balance'])
    except Exception as err:
        raise err
    
def getCurrentPrice(perpSymbol) :
    global kucoin
    try:
        ticker = kucoin.fetchTicker(perpSymbol)
    except BaseException as err:
        print("An error occured", err)
    return float(ticker["ask"])


#Dans le cas où vos clés API seraient mauvaises, cette fonction renverrait une erreur
try :
    test=getSolde();del test
except Exception as err:
    print(f"Problème de connexion à l'API : {err}")
    exit()
print(f"{str(datetime.now()).split('.')[0]} | Connexion à l'API Kucoin réussie.")
date=str(datetime.now()).split('.')[0]
print(f"{date} | Bot de sniping lancé avec {getSolde()} USDT, en attente de nouvelles paires...")

In [20]:
class KucoinAPI:
    def __init__(self) -> None:
        #self._credentials = {
        #    "apiKey":"62fa6d4f37a609000198c9ca",
        #    "secret":"7ab549c3-5fb9-46f6-8f5a-4ba63556102d",
        #    "password":"oHCsD5dDK&HrskYC8a9cYp9N@hgRRg9"
        #}#sandbox
        self._credentials = {
            "apiKey":"62a05a9ccf9fd60001e2ff9e",
            "secret":"578687f6-5420-46d7-9b7a-26a737713cfb",
            "password":"3FiF$4sR6p5Y9fPnSNijL?HaGcd@A7"
        }
        self._session = ccxt.kucoin(self._credentials)
        #self._session.set_sandbox_mode(True)

    
    def authentication_required(fn):
        """Annotation for methods that require auth."""
        def wrapped(self, *args, **kwargs):
            if not self._credentials or "apiKey" not in self._credentials or "secret" not in self._credentials or "password" not in self._credentials: 
                raise Exception(f"You must be authenticated to use this method {fn}") 
                #sys.exit()
            else:
                return fn(self, *args, **kwargs)
        return wrapped
        
    @authentication_required    
    def placeMarketOrder(self,symbol:str,side:str,amount:str)->dict:
        assert side.lower() in ["buy","sell"]
        trade_result = self._session.create_market_order(f'{symbol}/USDT', side.lower(), amount)
        self.last_order_id = trade_result['info']['orderId']
        self.last_order_timestamp = trade_result['timestamp']
        self.qty_in_position = trade_result['amount']
        self.last_trade_type = trade_result['side']
        return trade_result
    
    @authentication_required     
    def placeLimitOrder(self):
        pass  
    
    @authentication_required   
    def cancelOrder(self, orderId:str)->dict:
        return self._session.cancel_order(orderId)
    
    @authentication_required
    def getUSDTBalance(self)->float:
        try:
            for coin in self._session.fetchBalance()['info']['data']:
                if coin['currency']=='USDT' :
                    return float(coin['balance'])
        except Exception as err:
            raise err
        
    def getCurrentPrice(self,symbol:str)->float:
        try:
            return self._session.fetch_ticker(f'{symbol}/USDT')["ask"]
        except Exception as err:
            raise err
        
    def getAmountForSellingAll(self,symbol:str)->str:
        return self._session.amount_to_precision(f'{symbol}/USDT', self.qty_in_position)
    
    def getAmountForTrade(self, symbol:str,pct_wallet:int=1)->str:
        if self._session.load_markets()[f'{symbol}/USDT']['limits']['amount']['min']<self.getUSDTBalance()*pct_wallet/self.getCurrentPrice(f'{symbol}'):
            return self._session.amount_to_precision(f'{symbol}/USDT', self.getUSDTBalance()*pct_wallet/self.getCurrentPrice(f'{symbol}'))
        else:
            raise Exception(f"Not enough USDT to trade {symbol}") 
        


In [21]:
kucoin_client = KucoinAPI()

In [22]:
kucoin_client.getCurrentPrice('BTC')

20260.6

In [23]:
kucoin_client.getUSDTBalance()

120.14347388

In [24]:
kucoin_client.getAmountForTrade('BTC',0.01)

'0.00005932'

In [25]:
result = kucoin_client.placeMarketOrder('BTC','buy',kucoin_client.getAmountForTrade('BTC',0.01))

In [32]:
float(str(kucoin_client.qty_in_position))

5.931e-05

In [33]:
kucoin_client._session.amount_to_precision(f'BTC/USDT', kucoin_client.qty_in_position)

'0.00005931'

In [34]:
result = kucoin_client.placeMarketOrder('BTC','sell',kucoin_client._session.amount_to_precision(f'BTC/USDT', kucoin_client.qty_in_position))

In [35]:
print(result)

{'id': '630cecd492c7500001f44e96', 'clientOrderId': '490bd80d-53ad-4f68-a7e0-306075450c7a', 'info': {'orderId': '630cecd492c7500001f44e96'}, 'timestamp': 1661791444771, 'datetime': '2022-08-29T16:44:04.771Z', 'lastTradeTimestamp': None, 'symbol': 'BTC/USDT', 'type': 'market', 'side': 'sell', 'price': None, 'amount': 5.931e-05, 'cost': None, 'average': None, 'filled': None, 'remaining': None, 'status': None, 'fee': None, 'trades': None}


In [36]:
result

{'id': '630cecd492c7500001f44e96',
 'clientOrderId': '490bd80d-53ad-4f68-a7e0-306075450c7a',
 'info': {'orderId': '630cecd492c7500001f44e96'},
 'timestamp': 1661791444771,
 'datetime': '2022-08-29T16:44:04.771Z',
 'lastTradeTimestamp': None,
 'symbol': 'BTC/USDT',
 'type': 'market',
 'side': 'sell',
 'price': None,
 'amount': 5.931e-05,
 'cost': None,
 'average': None,
 'filled': None,
 'remaining': None,
 'status': None,
 'fee': None,
 'trades': None}